In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from datetime import timedelta
from openpyxl import load_workbook
from webdriver_manager.chrome import ChromeDriverManager
from riotwatcher import LolWatcher, ApiError

import selenium.webdriver.support.ui as ui
import time
import pandas as pd
import numpy as np
import os
import re
import threading

In [ ]:
# selenium driver
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values':
    {
        'notifications':2
    }
}
options.add_experimental_option('prefs', prefs)
options.add_argument("disable-infobars")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
wait = ui.WebDriverWait(driver,5)

In [ ]:
# team
divisions = ['LCS', 'LEC', 'LCK', 'LPL', 'LMS', 'CBLOL', 'LCL', 'LJL', 'LLN', 'CLS', 'OPL', 'GPL', 'TCL', 'VCS']
seasons = ['2018_Season', '2019_Season', '2020_Season']
team_names = []
team_wins = []
team_loses = []
team_winrates = []
team_streaks = []
team_divisions = []
team_seasons = []

for d in divisions:
    for s in seasons:
        website = "https://lol.gamepedia.com/{}/{}".format(d, s)
        driver.get(website)
        teams_info = driver.find_elements(By.XPATH, "//div[@class='inline-content'][1]//table[@class='wikitable2 standings']//tr")
        
        for t_f in teams_info:
            team_name = t_f.get_attribute('data-teamhighlight')
            if team_name != None:
                team_names.append(team_name)
                team_divisions.append(d)
                team_seasons.append(s.split('_')[0])

team_df = pd.DataFrame(zip(team_divisions, team_seasons, team_names), columns=['Division', 'Season', 'Team'])
team_df.drop_duplicates('Team', inplace = True)
team_df.to_excel("teams.xlsx")

In [ ]:
# player
unique_team_names = set(team_names)
player_names = []
u_team_names = []

for t_n in unique_team_names:
    website = "https://lol.gamepedia.com/{}".format(t_n.replace(' ','_'))
    driver.get(website)
    time.sleep(1)
    players_info = driver.find_elements(By.XPATH, "//table[@class='wikitable sortable team-members hoverable-rows team-members-current jquery-tablesorter']//tr")

    for p_i in players_info:
        try:
            print(p_i.text.split('\n')[1].split(' ')[0])
            player_name = p_i.text.split('\n')[1].split(' ')[0]
            player_names.append(player_name)
            u_team_names.append(t_n)
            
        except:
            pass

player_df = pd.DataFrame(zip(u_team_names, player_names), columns=['Team', 'Player'])
player_df.to_excel('player.xlsx')

In [ ]:
player_team_df = pd.merge(team_df, player_df, how='right', on='Team')
player_team_df.to_excel("player_team.xlsx")

In [2]:
# read player_team table
player_team = pd.read_excel('player_team.xlsx')
player_team = player_team.drop('Unnamed: 0', axis=1)
player_team

,Division,Season,Team,Player
0,LCS,2019,Team Liquid,Alphari
1,LCS,2019,Team Liquid,Santorin
2,LCS,2019,Team Liquid,Jensen
3,LCS,2019,Team Liquid,Tactical
4,LCS,2019,Team Liquid,CoreJJ
...,...,...,...,...
482,VCS,2020,Dashing Buffalo,Froggy
483,VCS,2020,Dashing Buffalo,Rby
484,VCS,2020,Dashing Buffalo,Arrietty
485,VCS,2020,Dashing Buffalo,Jad


In [3]:
server_division_dict = {"LCS":"na1", "LEC":"eun1", "LEC":"euw1", "LCK":"kr", "LPL":"jp1", "LMS":"jp1", "CBLOL":"br1", "LCL":"ru", "LJL":"jp1",
                        "LLA":"la1", "LLA":"la2", "OPL":"oc1", "TCL":"tr1"}

In [20]:
# get player matches info
api_key = 'RGAPI-112e5812-7a43-46a4-87d6-d1940a2b0862'
watcher = LolWatcher(api_key)
part_data= round(len(player_team)/5)

matches_info = []
for i in range(part_data, part_data*2):

    region = server_division_dict[player_team['Division'][i]]
    player_name = player_team['Player'][i]
    print(region)
    print(player_name)
    print(i)
    
    try:
        player = watcher.summoner.by_name(region, player_name)
        print(player)
        player_matches = watcher.match.matchlist_by_account(region, player['accountId'])['matches']

        j=0
        for player_match in player_matches:
            print('match----',j)
            j = j+1
            info_row = {}
            try:
                m = watcher.match.by_id(region, player_match['gameId'])
                info_row['Player_name'] = player_name
                info_row['gameId'] = m['gameId']
                info_row['platformId'] = m['platformId']
                info_row['gameDuration'] = m['gameDuration']
                info_row['gameVersion'] = m['gameVersion']

                blue_team = m['teams'][0]
                purple_team = m['teams'][1]
                info_row['blue_win'] = blue_team['win']
                info_row['blue_firstBlood'] = blue_team['firstBlood']
                info_row['blue_firstTower'] = blue_team['firstTower']
                info_row['blue_firstInhibitor'] = blue_team['firstInhibitor']
                info_row['blue_firstBaron'] = blue_team['firstBaron']
                info_row['blue_firstDragon'] = blue_team['firstDragon']
                info_row['blue_firstRiftHerald'] = blue_team['firstRiftHerald']
                info_row['blue_towerKills'] = blue_team['towerKills']
                info_row['blue_inhibitorKills'] = blue_team['inhibitorKills']
                info_row['blue_baronKills'] = blue_team['baronKills']
                info_row['blue_dragonKills'] = blue_team['dragonKills']
                info_row['blue_riftHeraldKills'] = blue_team['riftHeraldKills']

                info_row['purple_firstBlood'] = purple_team['firstBlood']
                info_row['purple_firstTower'] = purple_team['firstTower']
                info_row['purple_firstInhibitor'] = purple_team['firstInhibitor']
                info_row['purple_firstBaron'] = purple_team['firstBaron']
                info_row['purple_firstDragon'] = purple_team['firstDragon']
                info_row['purple_firstRiftHerald'] = purple_team['firstRiftHerald']
                info_row['purple_towerKills'] = purple_team['towerKills']
                info_row['purple_inhibitorKills'] = purple_team['inhibitorKills']
                info_row['purple_baronKills'] = purple_team['baronKills']
                info_row['purple_dragonKills'] = purple_team['dragonKills']
                info_row['purple_riftHeraldKills'] = purple_team['riftHeraldKills']

                ps = 0
                participants = m['participants']
                for p in participants:
                    print(ps)
                    ps = ps+1
                    if p['teamId'] == 100:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['blue_top_champion'] = p['championId']
                            info_row['blue_top_item0'] = p['stats']['item0']
                            info_row['blue_top_item1'] = p['stats']['item1']
                            info_row['blue_top_item2'] = p['stats']['item2']
                            info_row['blue_top_item3'] = p['stats']['item3']
                            info_row['blue_top_item4'] = p['stats']['item4']
                            info_row['blue_top_item5'] = p['stats']['item5']
                            info_row['blue_top_item6'] = p['stats']['item6']
                            info_row['blue_top_kills'] = p['stats']['kills']
                            info_row['blue_top_deaths'] = p['stats']['deaths']
                            info_row['blue_top_assists'] = p['stats']['assists']
        #                     info_row['blue_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_top_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_top_perk0'] = p['stats']['perk0']
                            info_row['blue_top_perk1'] = p['stats']['perk1']
                            info_row['blue_top_perk2'] = p['stats']['perk2']
                            info_row['blue_top_perk3'] = p['stats']['perk3']
                            info_row['blue_top_perk4'] = p['stats']['perk4']
                            info_row['blue_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['blue_jungle_champion'] = p['championId']
                            info_row['blue_jungle_item0'] = p['stats']['item0']
                            info_row['blue_jungle_item1'] = p['stats']['item1']
                            info_row['blue_jungle_item2'] = p['stats']['item2']
                            info_row['blue_jungle_item3'] = p['stats']['item3']
                            info_row['blue_jungle_item4'] = p['stats']['item4']
                            info_row['blue_jungle_item5'] = p['stats']['item5']
                            info_row['blue_jungle_item6'] = p['stats']['item6']
                            info_row['blue_jungle_kills'] = p['stats']['kills']
                            info_row['blue_jungle_deaths'] = p['stats']['deaths']
                            info_row['blue_jungle_assists'] = p['stats']['assists']
        #                     info_row['blue_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
    #                         info_row['blue_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
    #                         info_row['blue_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
    #                         info_row['blue_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_jungle_perk0'] = p['stats']['perk0']
                            info_row['blue_jungle_perk1'] = p['stats']['perk1']
                            info_row['blue_jungle_perk2'] = p['stats']['perk2']
                            info_row['blue_jungle_perk3'] = p['stats']['perk3']
                            info_row['blue_jungle_perk4'] = p['stats']['perk4']
                            info_row['blue_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['blue_mid_champion'] = p['championId']
                            info_row['blue_mid_item0'] = p['stats']['item0']
                            info_row['blue_mid_item1'] = p['stats']['item1']
                            info_row['blue_mid_item2'] = p['stats']['item2']
                            info_row['blue_mid_item3'] = p['stats']['item3']
                            info_row['blue_mid_item4'] = p['stats']['item4']
                            info_row['blue_mid_item5'] = p['stats']['item5']
                            info_row['blue_mid_item6'] = p['stats']['item6']
                            info_row['blue_mid_kills'] = p['stats']['kills']
                            info_row['blue_mid_deaths'] = p['stats']['deaths']
                            info_row['blue_mid_assists'] = p['stats']['assists']
        #                     info_row['blue_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_mid_perk0'] = p['stats']['perk0']
                            info_row['blue_mid_perk1'] = p['stats']['perk1']
                            info_row['blue_mid_perk2'] = p['stats']['perk2']
                            info_row['blue_mid_perk3'] = p['stats']['perk3']
                            info_row['blue_mid_perk4'] = p['stats']['perk4']
                            info_row['blue_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['blue_ad_champion'] = p['championId']
                            info_row['blue_ad_item0'] = p['stats']['item0']
                            info_row['blue_ad_item1'] = p['stats']['item1']
                            info_row['blue_ad_item2'] = p['stats']['item2']
                            info_row['blue_ad_item3'] = p['stats']['item3']
                            info_row['blue_ad_item4'] = p['stats']['item4']
                            info_row['blue_ad_item5'] = p['stats']['item5']
                            info_row['blue_ad_item6'] = p['stats']['item6']
                            info_row['blue_ad_kills'] = p['stats']['kills']
                            info_row['blue_ad_deaths'] = p['stats']['deaths']
                            info_row['blue_ad_assists'] = p['stats']['assists']
        #                     info_row['blue_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['blue_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['blue_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['blue_ad_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['blue_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_ad_perk0'] = p['stats']['perk0']
                            info_row['blue_ad_perk1'] = p['stats']['perk1']
                            info_row['blue_ad_perk2'] = p['stats']['perk2']
                            info_row['blue_ad_perk3'] = p['stats']['perk3']
                            info_row['blue_ad_perk4'] = p['stats']['perk4']
                            info_row['blue_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['blue_sup_champion'] = p['championId']
                            info_row['blue_sup_item0'] = p['stats']['item0']
                            info_row['blue_sup_item1'] = p['stats']['item1']
                            info_row['blue_sup_item2'] = p['stats']['item2']
                            info_row['blue_sup_item3'] = p['stats']['item3']
                            info_row['blue_sup_item4'] = p['stats']['item4']
                            info_row['blue_sup_item5'] = p['stats']['item5']
                            info_row['blue_sup_item6'] = p['stats']['item6']
                            info_row['blue_sup_kills'] = p['stats']['kills']
                            info_row['blue_sup_deaths'] = p['stats']['deaths']
                            info_row['blue_sup_assists'] = p['stats']['assists']
        #                     info_row['blue_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']                   
        #                     info_row['blue_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']                 
        #                     info_row['blue_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_sup_perk0'] = p['stats']['perk0']
                            info_row['blue_sup_perk1'] = p['stats']['perk1']
                            info_row['blue_sup_perk2'] = p['stats']['perk2']
                            info_row['blue_sup_perk3'] = p['stats']['perk3']
                            info_row['blue_sup_perk4'] = p['stats']['perk4']
                            info_row['blue_sup_perk5'] = p['stats']['perk5']

                    if p['teamId'] == 200:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['purple_top_champion'] = p['championId']
                            info_row['purple_top_item0'] = p['stats']['item0']
                            info_row['purple_top_item1'] = p['stats']['item1']
                            info_row['purple_top_item2'] = p['stats']['item2']
                            info_row['purple_top_item3'] = p['stats']['item3']
                            info_row['purple_top_item4'] = p['stats']['item4']
                            info_row['purple_top_item5'] = p['stats']['item5']
                            info_row['purple_top_item6'] = p['stats']['item6']
                            info_row['purple_top_kills'] = p['stats']['kills']
                            info_row['purple_top_deaths'] = p['stats']['deaths']
                            info_row['purple_top_assists'] = p['stats']['assists']
        #                     info_row['purple_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['purple_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['purple_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['purple_top_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['purple_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_top_perk0'] = p['stats']['perk0']
                            info_row['purple_top_perk1'] = p['stats']['perk1']
                            info_row['purple_top_perk2'] = p['stats']['perk2']
                            info_row['purple_top_perk3'] = p['stats']['perk3']
                            info_row['purple_top_perk4'] = p['stats']['perk4']
                            info_row['purple_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['purple_jungle_champion'] = p['championId']
                            info_row['purple_jungle_item0'] = p['stats']['item0']
                            info_row['purple_jungle_item1'] = p['stats']['item1']
                            info_row['purple_jungle_item2'] = p['stats']['item2']
                            info_row['purple_jungle_item3'] = p['stats']['item3']
                            info_row['purple_jungle_item4'] = p['stats']['item4']
                            info_row['purple_jungle_item5'] = p['stats']['item5']
                            info_row['purple_jungle_item6'] = p['stats']['item6']
                            info_row['purple_jungle_kills'] = p['stats']['kills']
                            info_row['purple_jungle_deaths'] = p['stats']['deaths']
                            info_row['purple_jungle_assists'] = p['stats']['assists']
        #                     info_row['purple_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_jungle_perk0'] = p['stats']['perk0']
                            info_row['purple_jungle_perk1'] = p['stats']['perk1']
                            info_row['purple_jungle_perk2'] = p['stats']['perk2']
                            info_row['purple_jungle_perk3'] = p['stats']['perk3']
                            info_row['purple_jungle_perk4'] = p['stats']['perk4']
                            info_row['purple_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['purple_mid_champion'] = p['championId']
                            info_row['purple_mid_item0'] = p['stats']['item0']
                            info_row['purple_mid_item1'] = p['stats']['item1']
                            info_row['purple_mid_item2'] = p['stats']['item2']
                            info_row['purple_mid_item3'] = p['stats']['item3']
                            info_row['purple_mid_item4'] = p['stats']['item4']
                            info_row['purple_mid_item5'] = p['stats']['item5']
                            info_row['purple_mid_item6'] = p['stats']['item6']
                            info_row['purple_mid_kills'] = p['stats']['kills']
                            info_row['purple_mid_deaths'] = p['stats']['deaths']
                            info_row['purple_mid_assists'] = p['stats']['assists']
        #                     info_row['purple_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_mid_perk0'] = p['stats']['perk0']
                            info_row['purple_mid_perk1'] = p['stats']['perk1']
                            info_row['purple_mid_perk2'] = p['stats']['perk2']
                            info_row['purple_mid_perk3'] = p['stats']['perk3']
                            info_row['purple_mid_perk4'] = p['stats']['perk4']
                            info_row['purple_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['purple_ad_champion'] = p['championId']
                            info_row['purple_ad_item0'] = p['stats']['item0']
                            info_row['purple_ad_item1'] = p['stats']['item1']
                            info_row['purple_ad_item2'] = p['stats']['item2']
                            info_row['purple_ad_item3'] = p['stats']['item3']
                            info_row['purple_ad_item4'] = p['stats']['item4']
                            info_row['purple_ad_item5'] = p['stats']['item5']
                            info_row['purple_ad_item6'] = p['stats']['item6']
                            info_row['purple_ad_kills'] = p['stats']['kills']
                            info_row['purple_ad_deaths'] = p['stats']['deaths']
                            info_row['purple_ad_assists'] = p['stats']['assists']
        #                     info_row['purple_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_ad_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_ad_perk0'] = p['stats']['perk0']
                            info_row['purple_ad_perk1'] = p['stats']['perk1']
                            info_row['purple_ad_perk2'] = p['stats']['perk2']
                            info_row['purple_ad_perk3'] = p['stats']['perk3']
                            info_row['purple_ad_perk4'] = p['stats']['perk4']
                            info_row['purple_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['purple_sup_champion'] = p['championId']
                            info_row['purple_sup_item0'] = p['stats']['item0']
                            info_row['purple_sup_item1'] = p['stats']['item1']
                            info_row['purple_sup_item2'] = p['stats']['item2']
                            info_row['purple_sup_item3'] = p['stats']['item3']
                            info_row['purple_sup_item4'] = p['stats']['item4']
                            info_row['purple_sup_item5'] = p['stats']['item5']
                            info_row['purple_sup_item6'] = p['stats']['item6']
                            info_row['purple_sup_kills'] = p['stats']['kills']
                            info_row['purple_sup_deaths'] = p['stats']['deaths']
                            info_row['purple_sup_assists'] = p['stats']['assists']
        #                     info_row['purple_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_sup_perk0'] = p['stats']['perk0']
                            info_row['purple_sup_perk1'] = p['stats']['perk1']
                            info_row['purple_sup_perk2'] = p['stats']['perk2']
                            info_row['purple_sup_perk3'] = p['stats']['perk3']
                            info_row['purple_sup_perk4'] = p['stats']['perk4']
                            info_row['purple_sup_perk5'] = p['stats']['perk5']

            except:
                time.sleep(3)
                m = watcher.match.by_id(region, player_match['gameId'])
                info_row['Player_name'] = player_name
                info_row['gameId'] = m['gameId']
                info_row['platformId'] = m['platformId']
                info_row['gameDuration'] = m['gameDuration']
                info_row['gameVersion'] = m['gameVersion']

                blue_team = m['teams'][0]
                purple_team = m['teams'][1]
                info_row['blue_win'] = blue_team['win']
                info_row['blue_firstBlood'] = blue_team['firstBlood']
                info_row['blue_firstTower'] = blue_team['firstTower']
                info_row['blue_firstInhibitor'] = blue_team['firstInhibitor']
                info_row['blue_firstBaron'] = blue_team['firstBaron']
                info_row['blue_firstDragon'] = blue_team['firstDragon']
                info_row['blue_firstRiftHerald'] = blue_team['firstRiftHerald']
                info_row['blue_towerKills'] = blue_team['towerKills']
                info_row['blue_inhibitorKills'] = blue_team['inhibitorKills']
                info_row['blue_baronKills'] = blue_team['baronKills']
                info_row['blue_dragonKills'] = blue_team['dragonKills']
                info_row['blue_riftHeraldKills'] = blue_team['riftHeraldKills']

                info_row['purple_firstBlood'] = purple_team['firstBlood']
                info_row['purple_firstTower'] = purple_team['firstTower']
                info_row['purple_firstInhibitor'] = purple_team['firstInhibitor']
                info_row['purple_firstBaron'] = purple_team['firstBaron']
                info_row['purple_firstDragon'] = purple_team['firstDragon']
                info_row['purple_firstRiftHerald'] = purple_team['firstRiftHerald']
                info_row['purple_towerKills'] = purple_team['towerKills']
                info_row['purple_inhibitorKills'] = purple_team['inhibitorKills']
                info_row['purple_baronKills'] = purple_team['baronKills']
                info_row['purple_dragonKills'] = purple_team['dragonKills']
                info_row['purple_riftHeraldKills'] = purple_team['riftHeraldKills']

                ps = 0
                participants = m['participants']
                for p in participants:
                    print(ps)
                    ps = ps+1
                    if p['teamId'] == 100:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['blue_top_champion'] = p['championId']
                            info_row['blue_top_item0'] = p['stats']['item0']
                            info_row['blue_top_item1'] = p['stats']['item1']
                            info_row['blue_top_item2'] = p['stats']['item2']
                            info_row['blue_top_item3'] = p['stats']['item3']
                            info_row['blue_top_item4'] = p['stats']['item4']
                            info_row['blue_top_item5'] = p['stats']['item5']
                            info_row['blue_top_item6'] = p['stats']['item6']
                            info_row['blue_top_kills'] = p['stats']['kills']
                            info_row['blue_top_deaths'] = p['stats']['deaths']
                            info_row['blue_top_assists'] = p['stats']['assists']
        #                     info_row['blue_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_top_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_top_perk0'] = p['stats']['perk0']
                            info_row['blue_top_perk1'] = p['stats']['perk1']
                            info_row['blue_top_perk2'] = p['stats']['perk2']
                            info_row['blue_top_perk3'] = p['stats']['perk3']
                            info_row['blue_top_perk4'] = p['stats']['perk4']
                            info_row['blue_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['blue_jungle_champion'] = p['championId']
                            info_row['blue_jungle_item0'] = p['stats']['item0']
                            info_row['blue_jungle_item1'] = p['stats']['item1']
                            info_row['blue_jungle_item2'] = p['stats']['item2']
                            info_row['blue_jungle_item3'] = p['stats']['item3']
                            info_row['blue_jungle_item4'] = p['stats']['item4']
                            info_row['blue_jungle_item5'] = p['stats']['item5']
                            info_row['blue_jungle_item6'] = p['stats']['item6']
                            info_row['blue_jungle_kills'] = p['stats']['kills']
                            info_row['blue_jungle_deaths'] = p['stats']['deaths']
                            info_row['blue_jungle_assists'] = p['stats']['assists']
        #                     info_row['blue_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
    #                         info_row['blue_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
    #                         info_row['blue_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
    #                         info_row['blue_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_jungle_perk0'] = p['stats']['perk0']
                            info_row['blue_jungle_perk1'] = p['stats']['perk1']
                            info_row['blue_jungle_perk2'] = p['stats']['perk2']
                            info_row['blue_jungle_perk3'] = p['stats']['perk3']
                            info_row['blue_jungle_perk4'] = p['stats']['perk4']
                            info_row['blue_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['blue_mid_champion'] = p['championId']
                            info_row['blue_mid_item0'] = p['stats']['item0']
                            info_row['blue_mid_item1'] = p['stats']['item1']
                            info_row['blue_mid_item2'] = p['stats']['item2']
                            info_row['blue_mid_item3'] = p['stats']['item3']
                            info_row['blue_mid_item4'] = p['stats']['item4']
                            info_row['blue_mid_item5'] = p['stats']['item5']
                            info_row['blue_mid_item6'] = p['stats']['item6']
                            info_row['blue_mid_kills'] = p['stats']['kills']
                            info_row['blue_mid_deaths'] = p['stats']['deaths']
                            info_row['blue_mid_assists'] = p['stats']['assists']
        #                     info_row['blue_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_mid_perk0'] = p['stats']['perk0']
                            info_row['blue_mid_perk1'] = p['stats']['perk1']
                            info_row['blue_mid_perk2'] = p['stats']['perk2']
                            info_row['blue_mid_perk3'] = p['stats']['perk3']
                            info_row['blue_mid_perk4'] = p['stats']['perk4']
                            info_row['blue_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['blue_ad_champion'] = p['championId']
                            info_row['blue_ad_item0'] = p['stats']['item0']
                            info_row['blue_ad_item1'] = p['stats']['item1']
                            info_row['blue_ad_item2'] = p['stats']['item2']
                            info_row['blue_ad_item3'] = p['stats']['item3']
                            info_row['blue_ad_item4'] = p['stats']['item4']
                            info_row['blue_ad_item5'] = p['stats']['item5']
                            info_row['blue_ad_item6'] = p['stats']['item6']
                            info_row['blue_ad_kills'] = p['stats']['kills']
                            info_row['blue_ad_deaths'] = p['stats']['deaths']
                            info_row['blue_ad_assists'] = p['stats']['assists']
        #                     info_row['blue_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['blue_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['blue_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['blue_ad_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['blue_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['blue_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['blue_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_ad_perk0'] = p['stats']['perk0']
                            info_row['blue_ad_perk1'] = p['stats']['perk1']
                            info_row['blue_ad_perk2'] = p['stats']['perk2']
                            info_row['blue_ad_perk3'] = p['stats']['perk3']
                            info_row['blue_ad_perk4'] = p['stats']['perk4']
                            info_row['blue_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['blue_sup_champion'] = p['championId']
                            info_row['blue_sup_item0'] = p['stats']['item0']
                            info_row['blue_sup_item1'] = p['stats']['item1']
                            info_row['blue_sup_item2'] = p['stats']['item2']
                            info_row['blue_sup_item3'] = p['stats']['item3']
                            info_row['blue_sup_item4'] = p['stats']['item4']
                            info_row['blue_sup_item5'] = p['stats']['item5']
                            info_row['blue_sup_item6'] = p['stats']['item6']
                            info_row['blue_sup_kills'] = p['stats']['kills']
                            info_row['blue_sup_deaths'] = p['stats']['deaths']
                            info_row['blue_sup_assists'] = p['stats']['assists']
        #                     info_row['blue_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['blue_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['blue_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['blue_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['blue_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['blue_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['blue_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['blue_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['blue_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['blue_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['blue_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['blue_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['blue_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['blue_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['blue_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['blue_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['blue_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['blue_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['blue_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['blue_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['blue_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['blue_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['blue_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['blue_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['blue_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['blue_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['blue_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['blue_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['blue_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['blue_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['blue_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['blue_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['blue_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['blue_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['blue_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['blue_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['blue_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['blue_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['blue_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['blue_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['blue_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']                   
        #                     info_row['blue_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']                 
        #                     info_row['blue_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['blue_sup_perk0'] = p['stats']['perk0']
                            info_row['blue_sup_perk1'] = p['stats']['perk1']
                            info_row['blue_sup_perk2'] = p['stats']['perk2']
                            info_row['blue_sup_perk3'] = p['stats']['perk3']
                            info_row['blue_sup_perk4'] = p['stats']['perk4']
                            info_row['blue_sup_perk5'] = p['stats']['perk5']

                    if p['teamId'] == 200:
                        if p['timeline']['lane'] == 'TOP':   
                            info_row['purple_top_champion'] = p['championId']
                            info_row['purple_top_item0'] = p['stats']['item0']
                            info_row['purple_top_item1'] = p['stats']['item1']
                            info_row['purple_top_item2'] = p['stats']['item2']
                            info_row['purple_top_item3'] = p['stats']['item3']
                            info_row['purple_top_item4'] = p['stats']['item4']
                            info_row['purple_top_item5'] = p['stats']['item5']
                            info_row['purple_top_item6'] = p['stats']['item6']
                            info_row['purple_top_kills'] = p['stats']['kills']
                            info_row['purple_top_deaths'] = p['stats']['deaths']
                            info_row['purple_top_assists'] = p['stats']['assists']
        #                     info_row['purple_top_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_top_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_top_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_top_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_top_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_top_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_top_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_top_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_top_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_top_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_top_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_top_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_top_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_top_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_top_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_top_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_top_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_top_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_top_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_top_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_top_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_top_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_top_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_top_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_top_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_top_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_top_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_top_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_top_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_top_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_top_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_top_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_top_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
    #                         info_row['purple_top_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
    #                         info_row['purple_top_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
        #                     info_row['purple_top_wardsPlaced'] = p['stats']['wardsPlaced']
        #                     info_row['purple_top_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_top_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_top_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_top_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_top_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_top_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_top_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_top_perk0'] = p['stats']['perk0']
                            info_row['purple_top_perk1'] = p['stats']['perk1']
                            info_row['purple_top_perk2'] = p['stats']['perk2']
                            info_row['purple_top_perk3'] = p['stats']['perk3']
                            info_row['purple_top_perk4'] = p['stats']['perk4']
                            info_row['purple_top_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'JUNGLE':   
                            info_row['purple_jungle_champion'] = p['championId']
                            info_row['purple_jungle_item0'] = p['stats']['item0']
                            info_row['purple_jungle_item1'] = p['stats']['item1']
                            info_row['purple_jungle_item2'] = p['stats']['item2']
                            info_row['purple_jungle_item3'] = p['stats']['item3']
                            info_row['purple_jungle_item4'] = p['stats']['item4']
                            info_row['purple_jungle_item5'] = p['stats']['item5']
                            info_row['purple_jungle_item6'] = p['stats']['item6']
                            info_row['purple_jungle_kills'] = p['stats']['kills']
                            info_row['purple_jungle_deaths'] = p['stats']['deaths']
                            info_row['purple_jungle_assists'] = p['stats']['assists']
        #                     info_row['purple_jungle_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_jungle_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_jungle_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_jungle_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_jungle_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_jungle_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_jungle_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_jungle_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_jungle_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_jungle_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_jungle_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_jungle_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_jungle_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_jungle_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_jungle_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_jungle_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_jungle_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_jungle_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_jungle_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_jungle_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_jungle_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_jungle_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_jungle_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_jungle_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_jungle_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_jungle_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_jungle_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_jungle_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_jungle_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_jungle_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_jungle_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_jungle_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_jungle_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_jungle_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_jungle_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_jungle_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_jungle_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_jungle_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_jungle_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_jungle_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_jungle_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_jungle_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_jungle_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_jungle_perk0'] = p['stats']['perk0']
                            info_row['purple_jungle_perk1'] = p['stats']['perk1']
                            info_row['purple_jungle_perk2'] = p['stats']['perk2']
                            info_row['purple_jungle_perk3'] = p['stats']['perk3']
                            info_row['purple_jungle_perk4'] = p['stats']['perk4']
                            info_row['purple_jungle_perk5'] = p['stats']['perk5']

                        if p['timeline']['lane'] == 'MIDDLE':   
                            info_row['purple_mid_champion'] = p['championId']
                            info_row['purple_mid_item0'] = p['stats']['item0']
                            info_row['purple_mid_item1'] = p['stats']['item1']
                            info_row['purple_mid_item2'] = p['stats']['item2']
                            info_row['purple_mid_item3'] = p['stats']['item3']
                            info_row['purple_mid_item4'] = p['stats']['item4']
                            info_row['purple_mid_item5'] = p['stats']['item5']
                            info_row['purple_mid_item6'] = p['stats']['item6']
                            info_row['purple_mid_kills'] = p['stats']['kills']
                            info_row['purple_mid_deaths'] = p['stats']['deaths']
                            info_row['purple_mid_assists'] = p['stats']['assists']
        #                     info_row['purple_mid_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_mid_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_mid_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_mid_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_mid_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_mid_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_mid_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_mid_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_mid_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_mid_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_mid_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_mid_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_mid_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_mid_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_mid_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_mid_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_mid_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_mid_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_mid_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_mid_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_mid_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_mid_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_mid_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_mid_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_mid_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_mid_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_mid_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_mid_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_mid_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_mid_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_mid_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_mid_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_mid_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_mid_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_mid_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_mid_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_mid_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_mid_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_mid_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_mid_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_mid_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_mid_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_mid_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_mid_perk0'] = p['stats']['perk0']
                            info_row['purple_mid_perk1'] = p['stats']['perk1']
                            info_row['purple_mid_perk2'] = p['stats']['perk2']
                            info_row['purple_mid_perk3'] = p['stats']['perk3']
                            info_row['purple_mid_perk4'] = p['stats']['perk4']
                            info_row['purple_mid_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_CARRY':   
                            info_row['purple_ad_champion'] = p['championId']
                            info_row['purple_ad_item0'] = p['stats']['item0']
                            info_row['purple_ad_item1'] = p['stats']['item1']
                            info_row['purple_ad_item2'] = p['stats']['item2']
                            info_row['purple_ad_item3'] = p['stats']['item3']
                            info_row['purple_ad_item4'] = p['stats']['item4']
                            info_row['purple_ad_item5'] = p['stats']['item5']
                            info_row['purple_ad_item6'] = p['stats']['item6']
                            info_row['purple_ad_kills'] = p['stats']['kills']
                            info_row['purple_ad_deaths'] = p['stats']['deaths']
                            info_row['purple_ad_assists'] = p['stats']['assists']
        #                     info_row['purple_ad_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_ad_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_ad_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_ad_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_ad_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_ad_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_ad_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_ad_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_ad_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_ad_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_ad_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_ad_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_ad_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_ad_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_ad_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_ad_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_ad_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_ad_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_ad_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_ad_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_ad_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_ad_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_ad_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_ad_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_ad_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_ad_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_ad_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_ad_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_ad_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_ad_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_ad_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_ad_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_ad_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_ad_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_ad_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_ad_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_ad_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_ad_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_ad_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_ad_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_ad_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_ad_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_ad_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_ad_perk0'] = p['stats']['perk0']
                            info_row['purple_ad_perk1'] = p['stats']['perk1']
                            info_row['purple_ad_perk2'] = p['stats']['perk2']
                            info_row['purple_ad_perk3'] = p['stats']['perk3']
                            info_row['purple_ad_perk4'] = p['stats']['perk4']
                            info_row['purple_ad_perk5'] = p['stats']['perk5']

                        if p['timeline']['role'] == 'DUO_SUPPORT':   
                            info_row['purple_sup_champion'] = p['championId']
                            info_row['purple_sup_item0'] = p['stats']['item0']
                            info_row['purple_sup_item1'] = p['stats']['item1']
                            info_row['purple_sup_item2'] = p['stats']['item2']
                            info_row['purple_sup_item3'] = p['stats']['item3']
                            info_row['purple_sup_item4'] = p['stats']['item4']
                            info_row['purple_sup_item5'] = p['stats']['item5']
                            info_row['purple_sup_item6'] = p['stats']['item6']
                            info_row['purple_sup_kills'] = p['stats']['kills']
                            info_row['purple_sup_deaths'] = p['stats']['deaths']
                            info_row['purple_sup_assists'] = p['stats']['assists']
        #                     info_row['purple_sup_largestKillingSpree'] = p['stats']['largestKillingSpree']
        #                     info_row['purple_sup_largestMultiKill'] = p['stats']['largestMultiKill']
        #                     info_row['purple_sup_killingSprees'] = p['stats']['killingSprees']
        #                     info_row['purple_sup_longestTimeSpentLiving'] = p['stats']['longestTimeSpentLiving']
        #                     info_row['purple_sup_doubleKills'] = p['stats']['doubleKills']
        #                     info_row['purple_sup_tripleKills'] = p['stats']['tripleKills']
        #                     info_row['purple_sup_quadraKills'] = p['stats']['quadraKills']
        #                     info_row['purple_sup_pentaKills'] = p['stats']['pentaKills']
        #                     info_row['purple_sup_unrealKills'] = p['stats']['unrealKills']
                            info_row['purple_sup_magicDamageDealt'] = p['stats']['magicDamageDealt']
                            info_row['purple_sup_physicalDamageDealt'] = p['stats']['physicalDamageDealt']
                            info_row['purple_sup_trueDamageDealt'] = p['stats']['trueDamageDealt']
        #                     info_row['purple_sup_largestCriticalStrike'] = p['stats']['largestCriticalStrike']
        #                     info_row['purple_sup_magicDamageDealtToChampions'] = p['stats']['magicDamageDealtToChampions']
        #                     info_row['purple_sup_physicalDamageDealtToChampions'] = p['stats']['physicalDamageDealtToChampions']
        #                     info_row['purple_sup_trueDamageDealtToChampions'] = p['stats']['trueDamageDealtToChampions']
                            info_row['purple_sup_totalHeal'] = p['stats']['totalHeal']
                            info_row['purple_sup_damageSelfMitigated'] = p['stats']['damageSelfMitigated']
                            info_row['purple_sup_damageDealtToObjectives'] = p['stats']['damageDealtToObjectives']
                            info_row['purple_sup_damageDealtToTurrets'] = p['stats']['damageDealtToTurrets']
                            info_row['purple_sup_visionScore'] = p['stats']['visionScore']
        #                     info_row['purple_sup_timeCCingOthers'] = p['stats']['timeCCingOthers']
        #                     info_row['purple_sup_magicalDamageTaken'] = p['stats']['magicalDamageTaken']
        #                     info_row['purple_sup_physicalDamageTaken'] = p['stats']['physicalDamageTaken']
        #                     info_row['purple_sup_trueDamageTaken'] = p['stats']['trueDamageTaken']
                            info_row['purple_sup_goldEarned'] = p['stats']['goldEarned']
        #                     info_row['purple_sup_goldSpent'] = p['stats']['goldSpent']
        #                     info_row['purple_sup_turretKills'] = p['stats']['turretKills']
        #                     info_row['purple_sup_inhibitorKills'] = p['stats']['inhibitorKills']
                            info_row['purple_sup_totalMinionsKilled'] = p['stats']['totalMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilled'] = p['stats']['neutralMinionsKilled']
        #                     info_row['purple_sup_neutralMinionsKilledTeamJungle'] = p['stats']['neutralMinionsKilledTeamJungle']
        #                     info_row['purple_sup_neutralMinionsKilledEnemyJungle'] = p['stats']['neutralMinionsKilledEnemyJungle']
        #                     info_row['purple_sup_totalTimeCrowdControlDealt'] = p['stats']['totalTimeCrowdControlDealt']
        #                     info_row['purple_sup_visionWardsBoughtInGame'] = p['stats']['visionWardsBoughtInGame']
        #                     info_row['purple_sup_sightWardsBoughtInGame'] = p['stats']['sightWardsBoughtInGame']
    #                         info_row['purple_sup_wardsPlaced'] = p['stats']['wardsPlaced']
    #                         info_row['purple_sup_wardsKilled'] = p['stats']['wardsKilled']
        #                     info_row['purple_sup_firstBloodKill'] = p['stats']['firstBloodKill']
        #                     info_row['purple_sup_firstBloodAssist'] = p['stats']['firstBloodAssist']
        #                     info_row['purple_sup_firstTowerKill'] = p['stats']['firstTowerKill']
        #                     info_row['purple_sup_firstTowerAssist'] = p['stats']['firstTowerAssist']
        #                     info_row['purple_sup_firstInhibitorKill'] = p['stats']['firstInhibitorKill']
        #                     info_row['purple_sup_firstInhibitorAssist'] = p['stats']['firstInhibitorAssist']
                            info_row['purple_sup_perk0'] = p['stats']['perk0']
                            info_row['purple_sup_perk1'] = p['stats']['perk1']
                            info_row['purple_sup_perk2'] = p['stats']['perk2']
                            info_row['purple_sup_perk3'] = p['stats']['perk3']
                            info_row['purple_sup_perk4'] = p['stats']['perk4']
                            info_row['purple_sup_perk5'] = p['stats']['perk5']

            matches_info.append(info_row)
    except:
        continue
    
    

matches_df = pd.DataFrame(matches_info)
matches_df.to_excel('matches2.xlsx')


euw1
Armut
97
{'id': '61aRkLVoaxhffZaSe6O9VoUJhdrtQeD7bzYPYQjE5gKo1a9L', 'accountId': '-EyXo90ReSRbCDSRFyKVrABKh412XabyB9m1scc3gIzJHiIxnnQth_b7', 'puuid': 'bMWEBmHHj8IiIXTyWNRGxwYufgJRKtIe1rLQ8m-4rxVTKTa-tnEY64N5jwEBKM61R5f2rVbaA2lpZA', 'name': 'Armut', 'profileIconId': 25, 'revisionDate': 1603892810000, 'summonerLevel': 55}
match---- 0
0
1
2
3
4
5
6
7
8
9
match---- 1
0
1
2
3
4
5
6
7
8
9
match---- 2
0
1
2
3
4
5
6
7
8
9
match---- 3
0
1
2
3
4
5
6
7
8
9
match---- 4
0
1
2
3
4
5
6
7
8
9
match---- 5
0
1
2
3
4
5
6
7
8
9
match---- 6
0
1
2
3
4
5
6
7
8
9
match---- 7
0
1
2
3
4
5
6
7
8
9
match---- 8
0
1
2
3
4
5
6
7
8
9
match---- 9
0
1
2
3
4
5
6
7
8
9
match---- 10
0
1
2
3
4
5
6
7
8
9
match---- 11
0
1
2
3
4
5
6
7
8
9
match---- 12
0
1
2
3
4
5
6
7
8
9
match---- 13
0
1
2
3
4
5
6
7
8
9
match---- 14
0
1
2
3
4
5
6
7
8
9
match---- 15
0
1
2
3
4
5
6
7
8
9
match---- 16
0
1
2
3
4
5
6
7
8
9
match---- 17
0
1
2
3
4
5
6
7
8
9
match---- 18
0
1
2
3
4
5
6
7
8
9
match---- 19
0
1
2
3
4
5
6
7
8
9
match---- 20
0
1
2
3
4


0
1
2
3
4
5
6
7
8
9
match---- 2
0
1
2
3
4
5
6
7
8
9
match---- 3
0
1
2
3
4
5
6
7
8
9
match---- 4
0
1
2
3
4
5
6
7
8
9
match---- 5
0
1
2
3
4
5
6
7
8
9
match---- 6
0
1
2
3
4
5
6
7
8
9
match---- 7
0
1
2
3
4
5
6
7
8
9
match---- 8
0
1
2
3
4
5
6
7
8
9
match---- 9
0
1
2
3
4
5
6
7
8
9
match---- 10
0
1
2
3
4
5
6
7
8
9
match---- 11
0
1
2
3
4
5
6
7
8
9
match---- 12
0
1
2
3
4
5
6
7
8
9
match---- 13
0
1
2
3
4
5
6
7
8
9
match---- 14
0
1
2
3
4
5
6
7
8
9
match---- 15
0
1
2
3
4
5
6
7
8
9
match---- 16
0
1
2
3
4
5
6
7
8
9
match---- 17
0
1
2
3
4
5
6
7
8
9
match---- 18
0
1
2
3
4
5
6
7
8
9
match---- 19
0
1
2
3
4
5
6
7
8
9
match---- 20
0
1
2
3
4
5
6
7
8
9
match---- 21
0
1
2
3
4
5
6
7
8
9
match---- 22
0
1
2
3
4
5
6
7
8
9
match---- 23
0
1
2
3
4
5
6
7
8
9
match---- 24
0
1
2
3
4
5
6
7
8
9
match---- 25
0
1
2
3
4
5
6
7
8
9
match---- 26
0
1
2
3
4
5
6
7
8
9
match---- 27
0
1
2
3
4
5
6
7
8
9
match---- 28
0
1
2
3
4
5
6
7
8
9
match---- 29
0
1
2
3
4
5
6
7
8
9
match---- 30
0
1
2
3
4
5
6
7
8
9
match---- 31
0
1
2
3
4
5
6
7
8


0
1
2
3
4
5
6
7
8
9
match---- 32
0
1
2
3
4
5
6
7
8
9
match---- 33
0
1
2
3
4
5
6
7
8
9
match---- 34
0
1
2
3
4
5
6
7
8
9
match---- 35
0
1
2
3
4
5
6
7
8
9
match---- 36
0
1
2
3
4
5
6
7
8
9
match---- 37
0
1
2
3
4
5
6
7
8
9
match---- 38
0
1
2
3
4
5
6
7
8
9
match---- 39
0
1
2
3
4
5
6
7
8
9
match---- 40
0
1
2
3
4
5
6
7
8
9
match---- 41
0
1
2
3
4
5
6
7
8
9
match---- 42
0
1
2
3
4
5
6
7
8
9
match---- 43
0
1
2
3
4
5
6
7
8
9
match---- 44
0
1
2
3
4
5
6
7
8
9
match---- 45
0
1
2
3
4
5
6
7
8
9
match---- 46
0
1
2
3
4
5
6
7
8
9
match---- 47
0
1
2
3
4
5
6
7
8
9
match---- 48
0
1
2
3
4
5
6
7
8
9
match---- 49
0
1
2
3
4
5
6
7
8
9
match---- 50
0
1
2
3
4
5
6
7
8
9
match---- 51
0
1
2
3
4
5
6
7
8
9
match---- 52
0
1
2
3
4
5
6
7
8
9
match---- 53
0
1
2
3
4
5
6
7
8
9
match---- 54
0
1
2
3
4
5
6
7
8
9
match---- 55
0
1
2
3
4
5
6
7
8
9
match---- 56
0
1
2
3
4
5
6
7
8
9
match---- 57
0
1
2
3
4
5
6
7
8
9
match---- 58
0
1
2
3
4
5
6
7
8
9
match---- 59
0
1
2
3
4
5
6
7
8
9
match---- 60
0
1
2
3
4
5
6
7
8
9
match---- 61
0
1
2
3
4


0
1
2
3
4
5
6
7
8
9
match---- 62
0
1
2
3
4
5
6
7
8
9
match---- 63
0
1
2
3
4
5
6
7
8
9
match---- 64
0
1
2
3
4
5
6
7
8
9
match---- 65
0
1
2
3
4
5
6
7
8
9
match---- 66
0
1
2
3
4
5
6
7
8
9
match---- 67
0
1
2
3
4
5
6
7
8
9
match---- 68
0
1
2
3
4
5
6
7
8
9
match---- 69
0
1
2
3
4
5
6
7
8
9
match---- 70
0
1
2
3
4
5
6
7
8
9
match---- 71
0
1
2
3
4
5
6
7
8
9
match---- 72
0
1
2
3
4
5
6
7
8
9
match---- 73
0
1
2
3
4
5
6
7
8
9
match---- 74
0
1
2
3
4
5
6
7
8
9
match---- 75
0
1
2
3
4
5
6
7
8
9
match---- 76
0
1
2
3
4
5
6
7
8
9
match---- 77
0
1
2
3
4
5
6
7
8
9
match---- 78
0
1
2
3
4
5
6
7
8
9
match---- 79
0
1
2
3
4
5
6
7
8
9
match---- 80
0
1
2
3
4
5
6
7
8
9
match---- 81
0
1
2
3
4
5
6
7
8
9
match---- 82
0
1
2
3
4
5
6
7
8
9
match---- 83
0
1
2
3
4
5
6
7
8
9
match---- 84
0
1
2
3
4
5
6
7
8
9
match---- 85
0
1
2
3
4
5
6
7
8
9
match---- 86
0
1
2
3
4
5
6
7
8
9
match---- 87
0
1
2
3
4
5
6
7
8
9
match---- 88
0
1
2
3
4
5
6
7
8
9
match---- 89
0
1
2
3
4
5
6
7
8
9
match---- 90
0
1
2
3
4
5
6
7
8
9
match---- 91
0
1
2
3
4


0
1
2
3
4
5
6
7
8
9
match---- 92
0
1
2
3
4
5
6
7
8
9
match---- 93
0
1
2
3
4
5
6
7
8
9
match---- 94
0
1
2
3
4
5
6
7
8
9
match---- 95
0
1
2
3
4
5
6
7
8
9
match---- 96
0
1
2
3
4
5
6
7
8
9
match---- 97
0
1
2
3
4
5
6
7
8
9
match---- 98
0
1
2
3
4
5
6
7
8
9
match---- 99
0
1
2
3
4
5
6
7
8
9
kr
Bang
107
{'id': 'Ln8Ehi4-DD7yKYzGYlkc12dXcoZkhuoo8fZ-NnT0Wy2whdE', 'accountId': 'qNOwvHPPuV4fevvTzKupkVZhEih5jelblrmYztkGDAPgYhbMhNqshsju', 'puuid': 'T1kzuK_DoTRz0u8V6yhPYVB85HE0XEAfpPOH4rHRg8AHnIPnGUg9yk6hz1M40bSLcSU4pxVFrOkTRA', 'name': 'Bang', 'profileIconId': 1297, 'revisionDate': 1607699420000, 'summonerLevel': 120}
match---- 0
0
1
2
3
4
5
6
7
8
9
match---- 1
0
1
2
3
4
5
6
7
8
9
match---- 2
0
1
2
3
4
5
6
7
8
9
match---- 3
0
1
2
3
4
5
6
7
8
9
match---- 4
0
1
2
3
4
5
6
7
8
9
match---- 5
0
1
2
3
4
5
6
7
8
9
match---- 6
0
1
2
3
4
5
6
7
8
9
match---- 7
0
1
2
3
4
5
6
7
8
9
match---- 8
0
1
2
3
4
5
6
7
8
9
match---- 9
0
1
2
3
4
5
6
7
8
9
match---- 10
0
1
2
3
4
5
6
7
8
9
match---- 11
0
1
2
3
4
5
6
7
8
9
matc

...

0
1
2
3
4
5
6
7
8
9
match---- 11
0
1
2
3
4
5
6
7
8
9
match---- 12
0
1
2
3
4
5
6
7
8
9
match---- 13
0
1
2
3
4
5
6
7
8
9
match---- 14
0
1
2
3
4
5
6
7
8
9
match---- 15
0
1
2
3
4
5
6
7
8
9
match---- 16
0
1
2
3
4
5
6
7
8
9
match---- 17
0
1
2
3
4
5
6
7
8
9
match---- 18
0
1
2
3
4
5
6
7
8
9
match---- 19
0
1
2
3
4
5
6
7
8
9
match---- 20
0
1
2
3
4
5
6
7
8
9
match---- 21
0
1
2
3
4
5
6
7
8
9
match---- 22
0
1
2
3
4
5
6
7
8
9
match---- 23
0
1
2
3
4
5
6
7
8
9
match---- 24
0
1
2
3
4
5
6
7
8
9
match---- 25
0
1
2
3
4
5
6
7
8
9
match---- 26
0
1
2
3
4
5
6
7
8
9
match---- 27
0
1
2
3
4
5
6
7
8
9
match---- 28
0
1
2
3
4
5
6
7
8
9
match---- 29
0
1
2
3
4
5
6
7
8
9
match---- 30
0
1
2
3
4
5
6
7
8
9
match---- 31
0
1
2
3
4
5
6
7
8
9
match---- 32
0
1
2
3
4
5
6
7
8
9
match---- 33
0
1
2
3
4
5
6
7
8
9
match---- 34
0
1
2
3
4
5
6
7
8
9
match---- 35
0
1
2
3
4
5
6
7
8
9
match---- 36
0
1
2
3
4
5
6
7
8
9
match---- 37
0
1
2
3
4
5
6
7
8
9
match---- 38
0
1
2
3
4
5
6
7
8
9
match---- 39
0
1
2
3
4
5
6
7
8
9
match---- 40
0
1
2
3
4


0
1
2
3
4
5
6
7
8
9
match---- 31
0
1
2
3
4
5
6
7
8
9
match---- 32
0
1
2
3
4
5
6
7
8
9
match---- 33
0
1
2
3
4
5
6
7
8
9
match---- 34
0
1
2
3
4
5
6
7
8
9
match---- 35
0
1
2
3
4
5
6
7
8
9
match---- 36
0
1
2
3
4
5
6
7
8
9
match---- 37
0
1
2
3
4
5
6
7
8
9
match---- 38
0
1
2
3
4
5
6
7
8
9
match---- 39
0
1
2
3
4
5
6
7
8
9
match---- 40
0
1
2
3
4
5
6
7
8
9
match---- 41
0
1
2
3
4
5
6
7
8
9
match---- 42
0
1
2
3
4
5
6
7
8
9
match---- 43
0
1
2
3
4
5
6
7
8
9
match---- 44
0
1
2
3
4
5
6
7
8
9
match---- 45
0
1
2
3
4
5
6
7
8
9
match---- 46
0
1
2
3
4
5
6
7
8
9
match---- 47
0
1
2
3
4
5
6
7
8
9
match---- 48
0
1
2
3
4
5
6
7
8
9
match---- 49
0
1
2
3
4
5
6
7
8
9
match---- 50
0
1
2
3
4
5
6
7
8
9
match---- 51
0
1
2
3
4
5
6
7
8
9
match---- 52
0
1
2
3
4
5
6
7
8
9
match---- 53
0
1
2
3
4
5
6
7
8
9
match---- 54
0
1
2
3
4
5
6
7
8
9
match---- 55
0
1
2
3
4
5
6
7
8
9
match---- 56
0
1
2
3
4
5
6
7
8
9
match---- 57
0
1
2
3
4
5
6
7
8
9
match---- 58
0
1
2
3
4
5
6
7
8
9
match---- 59
0
1
2
3
4
5
6
7
8
9
match---- 60
0
1
2
3
4


0
1
2
3
4
5
6
7
8
9
match---- 41
0
1
2
3
4
5
6
7
8
9
match---- 42
0
1
2
3
4
5
6
7
8
9
match---- 43
0
1
2
3
4
5
6
7
8
9
match---- 44
0
1
2
3
4
5
6
7
8
9
match---- 45
0
1
2
3
4
5
6
7
8
9
match---- 46
0
1
2
3
4
5
6
7
8
9
match---- 47
0
1
2
3
4
5
6
7
8
9
match---- 48
0
1
2
3
4
5
6
7
8
9
match---- 49
0
1
2
3
4
5
6
7
8
9
match---- 50
0
1
2
3
4
5
6
7
8
9
match---- 51
0
1
2
3
4
5
6
7
8
9
match---- 52
0
1
2
3
4
5
6
7
8
9
match---- 53
0
1
2
3
4
5
6
7
8
9
match---- 54
0
1
2
3
4
5
6
7
8
9
match---- 55
0
1
2
3
4
5
6
7
8
9
match---- 56
0
1
2
3
4
5
6
7
8
9
match---- 57
0
1
2
3
4
5
6
7
8
9
match---- 58
0
1
2
3
4
5
6
7
8
9
match---- 59
0
1
2
3
4
5
6
7
8
9
match---- 60
0
1
2
3
4
5
6
7
8
9
match---- 61
0
1
2
3
4
5
6
7
8
9
match---- 62
0
1
2
3
4
5
6
7
8
9
match---- 63
0
1
2
3
4
5
6
7
8
9
match---- 64
0
1
2
3
4
5
6
7
8
9
match---- 65
0
1
2
3
4
5
6
7
8
9
match---- 66
0
1
2
3
4
5
6
7
8
9
match---- 67
0
1
2
3
4
5
6
7
8
9
match---- 68
0
1
2
3
4
5
6
7
8
9
match---- 69
0
1
2
3
4
5
6
7
8
9
match---- 70
0
1
2
3
4


0
1
2
3
4
5
6
7
8
9
match---- 53
0
1
2
3
4
5
6
7
8
9
match---- 54
0
1
2
3
4
5
6
7
8
9
match---- 55
0
1
2
3
4
5
6
7
8
9
match---- 56
0
1
2
3
4
5
6
7
8
9
match---- 57
0
1
2
3
4
5
6
7
8
9
match---- 58
0
1
2
3
4
5
6
7
8
9
match---- 59
0
1
2
3
4
5
6
7
8
9
match---- 60
0
1
2
3
4
5
6
7
8
9
match---- 61
0
1
2
3
4
5
6
7
8
9
match---- 62
0
1
2
3
4
5
6
7
8
9
match---- 63
0
1
2
3
4
5
6
7
8
9
match---- 64
0
1
2
3
4
5
6
7
8
9
match---- 65
0
1
2
3
4
5
6
7
8
9
match---- 66
0
1
2
3
4
5
6
7
8
9
match---- 67
0
1
2
3
4
5
6
7
8
9
match---- 68
0
1
2
3
4
5
6
7
8
9
match---- 69
0
1
2
3
4
5
6
7
8
9
match---- 70
0
1
2
3
4
5
6
7
8
9
match---- 71
0
1
2
3
4
5
6
7
8
9
match---- 72
0
1
2
3
4
5
6
7
8
9
match---- 73
0
1
2
3
4
5
6
7
8
9
match---- 74
0
1
2
3
4
5
6
7
8
9
match---- 75
0
1
2
3
4
5
6
7
8
9
match---- 76
0
1
2
3
4
5
6
7
8
9
match---- 77
0
1
2
3
4
5
6
7
8
9
match---- 78
0
1
2
3
4
5
6
7
8
9
match---- 79
0
1
2
3
4
5
6
7
8
9
match---- 80
0
1
2
3
4
5
6
7
8
9
match---- 81
0
1
2
3
4
5
6
7
8
9
match---- 82
0
1
2
3
4


0
1
2
3
4
5
6
7
8
9
match---- 11
0
1
2
3
4
5
6
7
8
9
match---- 12
0
1
2
3
4
5
6
7
8
9
match---- 13
0
1
2
3
4
5
6
7
8
9
match---- 14
0
1
2
3
4
5
6
7
8
9
match---- 15
0
1
2
3
4
5
6
7
8
9
match---- 16
0
1
2
3
4
5
6
7
8
9
match---- 17
0
1
2
3
4
5
6
7
8
9
match---- 18
0
1
2
3
4
5
6
7
8
9
match---- 19
0
1
2
3
4
5
6
7
8
9
match---- 20
0
1
2
3
4
5
6
7
8
9
match---- 21
0
1
2
3
4
5
6
7
8
9
match---- 22
0
1
2
3
4
5
6
7
8
9
match---- 23
0
1
2
3
4
5
6
7
8
9
match---- 24
0
1
2
3
4
5
6
7
8
9
match---- 25
0
1
2
3
4
5
6
7
8
9
match---- 26
0
1
2
3
4
5
6
7
8
9
match---- 27
0
1
2
3
4
5
6
7
8
9
match---- 28
0
1
2
3
4
5
6
7
8
9
match---- 29
0
1
2
3
4
5
6
7
8
9
match---- 30
0
1
2
3
4
5
6
7
8
9
match---- 31
0
1
2
3
4
5
6
7
8
9
match---- 32
0
1
2
3
4
5
6
7
8
9
match---- 33
0
1
2
3
4
5
6
7
8
9
match---- 34
0
1
2
3
4
5
6
7
8
9
match---- 35
0
1
2
3
4
5
6
7
8
9
match---- 36
0
1
2
3
4
5
6
7
8
9
match---- 37
0
1
2
3
4
5
6
7
8
9
match---- 38
0
1
2
3
4
5
6
7
8
9
match---- 39
0
1
2
3
4
5
6
7
8
9
match---- 40
0
1
2
3
4


In [ ]:
# champions
champions_version = watcher.data_dragon.versions_for_region(region)['n']['champion']
# print(champions_version)
champ_list = watcher.data_dragon.champions(champions_version)

champ_dict = {}
for key in champ_list['data']:
    row = champ_list['data'][key]
    champ_dict[row['key']] = row['id']

champ_dict

In [ ]:
# items
items_version = watcher.data_dragon.versions_for_region(region)['n']['item']
items_list = watcher.data_dragon.items(items_version)

items_dict = {}
for key in items_list['data']:
    row = items_list['data'][key]
    items_dict[key] = row['name']

items_dict

In [ ]:
# runes
runes_version = watcher.data_dragon.versions_for_region(region)['n']['rune']
runes_list = watcher.data_dragon.runes_reforged(runes_version)
# print(runes_list[0]['slots'][0]['runes'][0]['id'])

runes_path_dict = {}
runes_dict = {}

for key in runes_list:
    runes_path_dict[key['id']] = key['name']

for key in runes_list:
    for slot in key['slots']:
        for runes in slot['runes']:
            runes_dict[runes['id']] = runes['name']

runes_dict
# items_dict

# For dear 采辰<3:

In [16]:
api_key = 'RGAPI-112e5812-7a43-46a4-87d6-d1940a2b0862'
watcher = LolWatcher(api_key)

region = 'na1'
player_name = 'Alphari'
print(region)
print(player_name)

player = watcher.summoner.by_name(region, player_name)
print(player)
player_matches = watcher.match.matchlist_by_account(region, player['accountId'])['matches']
# player_matches[0]
m = watcher.match.by_id(region, player_matches[17]['gameId'])
m

na1
Alphari
{'id': 'Hc5MUFFPw9WZXoSn5IikT7Ft8D8kM1SIF4j0id79M6mGbKM', 'accountId': 'NCBIzfbUDG0zqo8BSc07HZys6SFj3mgLkBZFTogk7XgbvTE', 'puuid': '4pPz6RfoBkIY8vQRaNdAO2JVPCCkQUCVRnOt6NWKVPMexjAXEKNJv2YjvmdHWvoCtbxyS9r8r3w30w', 'name': 'Alphari', 'profileIconId': 1296, 'revisionDate': 1590175106000, 'summonerLevel': 25}


{'gameId': 3230070596,
 'platformId': 'NA1',
 'gameCreation': 1575936992082,
 'gameDuration': 1324,
 'queueId': 850,
 'mapId': 11,
 'seasonId': 13,
 'gameVersion': '9.23.299.3089',
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'teams': [{'teamId': 100,
   'win': 'Win',
   'firstBlood': True,
   'firstTower': False,
   'firstInhibitor': True,
   'firstBaron': False,
   'firstDragon': True,
   'firstRiftHerald': True,
   'towerKills': 10,
   'inhibitorKills': 2,
   'baronKills': 0,
   'dragonKills': 3,
   'vilemawKills': 0,
   'riftHeraldKills': 1,
   'dominionVictoryScore': 0,
   'bans': []},
  {'teamId': 200,
   'win': 'Fail',
   'firstBlood': False,
   'firstTower': True,
   'firstInhibitor': False,
   'firstBaron': False,
   'firstDragon': False,
   'firstRiftHerald': False,
   'towerKills': 1,
   'inhibitorKills': 0,
   'baronKills': 0,
   'dragonKills': 0,
   'vilemawKills': 0,
   'riftHeraldKills': 0,
   'dominionVictoryScore': 0,
   'bans': []}],
 'participants': [{'parti

In [ ]:
champ_dict['22']

In [ ]:
api_key = 'RGAPI-64ce7055-d90b-467a-86af-869b1db009d1'
watcher = LolWatcher(api_key)
region = 'na1'
player_name = 'Sneaky'
player = watcher.summoner.by_name(region, player_name)
print(player)

In [ ]:
teams = []
teams_info = driver.find_elements(By.XPATH, "//div[@class='inline-content'][1]//table[@class='wikitable2 standings']//tr")
for t_f in teams_info[6:len(teams_info)]:
    team_info = t_f.text.replace('\u2060\u2060','').split(' ', 1)[1].split('/n')
    team_names.append(team_info[0])
    
    team_record = team_info[1].split(' ')
    teams_win.append(team_record[0])
    teams_lose.append(team_record[2])
    teams_winrate.append(team_record[3])
    teams_streak.append(team_record[4])